In [ ]:
import pandas as pd
import ftfy

In [ ]:
df = pd.read_csv('train.csv', encoding='latin-1')

Data preparation

resolve corrupted characters

In [ ]:
converted_text = df["text"].apply(ftfy.fix_text)
print((converted_text != df["text"]).sum())
indices = df[df["text"] != converted_text].index
df.loc[indices, "text"]

In [ ]:
converted_text.loc[indices]

People use angle brackets `< >` to indicate:
- **Actions** they're performing
- **Reactions** to something
- **Emotions** or internal states
- **Comedic effect**

=> covert the `&lt;gasp!&gt;` to `<gasp!>` (could give the model an idea of the sentiment in the text)

In [ ]:
# Install: pip install emoji
import emoji
import html

text_converted = df["text"].apply(emoji.demojize)
# Result: "Fire in building :fire: :face_screaming_in_fear:"
# Direct write using magic command
converted_text = df["text"].apply(emoji.demojize).apply(html.unescape)# use line 2066 for html example
# take emojis out because they are few -> count how many entries are modified after applying emoji.demojize
print((converted_text != df["text"]).sum())

df["text"] = converted_text

t.co is the URL shortener service used by X (formerly Twitter) to shorten all links shared on its platform. It serves several key functions: it protects users from malicious websites by scanning links for harmful content, it provides analytics on user engagement, and it simplifies the display of long URLs to save space in posts. When you click a t.co link, you are first sent to the t.co domain, which then redirects you to the original, full URL
does not provide a meaningfull information since the link is build like `http://t.co/*id*` => make a token for the link (preserves the context of a link existing)

In [ ]:
df["text"] = df["text"].replace(r"http\S+", "<LINK>", regex=True)

reduce the mentions but keep the context of a mention

In [ ]:
df["text"] = df["text"].replace(r"@\S+", "<MENTION>", regex=True)

remove duplicates after tokanization because, links or mentions, might make the value distinct but the overall text is the same

In [ ]:
# check for duplicates and remove them
print("duplicated: ", df["text"].duplicated().sum())
print("df size: ", df.size)
df = df.drop_duplicates(subset=["text"])
print("duplicated: ", df["text"].duplicated().sum())
print("df size: ", df.size)

remove multiple spaces and new lines (less noise for classification)

In [ ]:
df["text"] = df["text"].replace(r'\s+', ' ', regex=True)

In [ ]:
df["text"].to_csv('output.csv', index=False)

In [ ]:
print((df["target"] == 1).sum())
print((df["target"] == 0).sum())

Hello!